In [1]:
import pandas as pd
import numpy as np

from scipy.spatial import distance

In [2]:
from wikipedia2vec import Wikipedia2Vec
wiki2vec = Wikipedia2Vec.load('/Users/gx/Data61/Emerging Digital Technologies/sourceFiles/enwiki_20180420_100d.pkl')

import gensim
# Load Google New Corpus W2V vectors
model = gensim.models.KeyedVectors.load_word2vec_format('/Users/gx/Data61/Emerging Digital Technologies/sourceFiles/enwiki_20180420_100d.txt')
vocab = model.vocab.keys()
df_vocab = pd.Series(list(vocab))


In [3]:
basic_research_vec = wiki2vec.get_entity_vector('Basic research')
r_and_d_vec = wiki2vec.get_entity_vector('Research and development')
research_vec = wiki2vec.get_entity_vector('Research')

business_vec = wiki2vec.get_entity_vector('Business')
industry_vec = wiki2vec.get_entity_vector('Industry')

In [4]:
df_valid_tech = pd.read_csv('Datasets/ET30k_Technology_Embeddings_Vectors.csv')
df_deep_tech = df_valid_tech[['Technology']].copy()
df_valid_tech = df_valid_tech.set_index('Technology')

In [8]:
simi_to_basic_research_list = []
simi_to_r_and_d_list = []
simi_to_research_list = []
simi_to_business_list = []
simi_to_industry_list = []

for tech in df_deep_tech['Technology'].tolist():
    simi_to_basic_research_list.append(1-distance.cosine(df_valid_tech.loc[tech,:],basic_research_vec))
    simi_to_r_and_d_list.append(1-distance.cosine(df_valid_tech.loc[tech,:],r_and_d_vec))
    simi_to_research_list.append(1-distance.cosine(df_valid_tech.loc[tech,:],research_vec))
    simi_to_business_list.append(1-distance.cosine(df_valid_tech.loc[tech,:],business_vec))
    simi_to_industry_list.append(1-distance.cosine(df_valid_tech.loc[tech,:],industry_vec))
    
df_deep_tech['Similar_to_Basic_Research'] = simi_to_basic_research_list
df_deep_tech['Similar_to_R_&_D'] = simi_to_r_and_d_list
df_deep_tech['Similar_to_Research'] = simi_to_research_list
df_deep_tech['Similar_to_Business'] = simi_to_business_list
df_deep_tech['Similar_to_Industry'] = simi_to_industry_list

df_deep_tech['Average_Research'] = df_deep_tech[['Similar_to_Basic_Research','Similar_to_R_&_D','Similar_to_Research']].mean(axis = 1)
df_deep_tech['Average_Business'] = df_deep_tech[['Similar_to_Business','Similar_to_Industry']].mean(axis = 1)

df_deep_tech['Researchy'] = df_deep_tech['Average_Research'].rank(pct=True)
df_deep_tech['Businessy'] = df_deep_tech['Average_Business'].rank(pct=True)

df_deep_tech['DeepTech_Index'] = df_deep_tech['Businessy'].apply(lambda x: 1-x)
df_deep_tech['DeepTech_Index'] = df_deep_tech['Researchy'] + df_deep_tech['DeepTech_Index']
df_deep_tech['DeepTech_Index'] = df_deep_tech['DeepTech_Index'].rank(pct=True)



In [9]:
df_deep_tech.sort_values('DeepTech_Index', ascending = False)

,Technology,Similar_to_Basic_Research,Similar_to_R_&_D,Similar_to_Research,Similar_to_Business,Similar_to_Industry,Average_Research,Average_Business,Researchy,Businessy,DeepTech_Index
4059,NIH Toolbox,0.513405,0.423508,0.420375,0.154678,0.075301,0.452429,0.114989,0.972374,0.007251,1.000000
12029,Subgroup analysis,0.505585,0.409123,0.438076,0.151658,0.141586,0.450928,0.146622,0.970667,0.025920,0.999967
19734,Beckman Institute for Advanced Science and Tec...,0.517972,0.423308,0.384697,0.121627,0.160725,0.441992,0.141176,0.960824,0.020999,0.999934
23808,DNA sequencing theory,0.476094,0.457657,0.427600,0.173479,0.136627,0.453784,0.155053,0.973653,0.036124,0.999902
10644,Reproducibility,0.517513,0.364539,0.454917,0.133199,0.168073,0.445656,0.150636,0.965352,0.029989,0.999869
...,...,...,...,...,...,...,...,...,...,...,...
18908,Petroleum industry in Canada,0.231882,0.276485,0.235198,0.390435,0.461177,0.247855,0.425806,0.076645,0.983037,0.000164
27357,Soft plastic bait,0.118306,0.291212,0.238696,0.369640,0.387301,0.216071,0.378470,0.023197,0.937726,0.000131
6575,Cryptocurrency wallet,0.215341,0.280785,0.200400,0.453327,0.336764,0.232175,0.395045,0.044885,0.960004,0.000098
24798,Lifeway,0.138555,0.155723,0.290523,0.413545,0.346371,0.194934,0.379958,0.008924,0.939465,0.000066


In [10]:
df_deep_tech.to_csv('Datasets/ET30k_Deep_Tech_Index.csc', index = False)